# Text Classification
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. Please check the pdf file for more details.*

In this exercise you will:
    
- implement a of spam classifier with **Naive Bayes method** for real world email messages
- learn the **training and testing phase** for Naive Bayes classifier  
- get an idea of the **precision-recall** tradeoff

In [1]:
# some basic imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [10]:
# ham_train contains the occurrences of each word in ham emails. 1-by-N vector
ham_train = np.loadtxt('ham_train.csv', delimiter=',')
# spam_train contains the occurrences of each word in spam emails. 1-by-N vector
spam_train = np.loadtxt('spam_train.csv', delimiter=',')

# N is the size of vocabulary.
N = ham_train.shape[0]
# There 9034 ham emails and 3372 spam emails in the training samples
num_ham_train = 9034
num_spam_train = 3372
# Do smoothing
x = np.vstack([ham_train, spam_train]) + 1

# ham_test contains the occurences of each word in each ham test email. P-by-N vector, with P is number of ham test emails.
i,j,ham_test = np.loadtxt('ham_test.txt').T
i = i.astype(np.int)
j = j.astype(np.int)
ham_test_tight = scipy.sparse.coo_matrix((ham_test, (i - 1, j - 1)))
ham_test = scipy.sparse.csr_matrix((ham_test_tight.shape[0], ham_train.shape[0]))
ham_test[:, 0:ham_test_tight.shape[1]] = ham_test_tight
# spam_test contains the occurences of each word in each spam test email. Q-by-N vector, with Q is number of spam test emails.
i,j,spam_test = np.loadtxt('spam_test.txt').T
i = i.astype(np.int)
j = j.astype(np.int)
spam_test_tight = scipy.sparse.csr_matrix((spam_test, (i - 1, j - 1)))
spam_test = scipy.sparse.csr_matrix((spam_test_tight.shape[0], spam_train.shape[0]))
spam_test[:, 0:spam_test_tight.shape[1]] = spam_test_tight


/Users/Alan/anaconda3/lib/python3.6/site-packages/scipy/sparse/_index.py:118: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


ok


## Now let's implement a ham/spam email classifier. Please refer to the PDF file for details

In [ ]:
from likelihood import likelihood
# TODO
# Implement a ham/spam email classifier, and calculate the accuracy of your classifier

# Hint: you can directly do matrix multiply between scipy.sparse.coo_matrix and numpy.array.
# Specifically, you can use sparse_matrix * np_array to do this. Note that when you use "*" operator
# between numpy array, this is typically an elementwise multiply.

# begin answer
# for a)
import re
lk = likelihood(x)
r = np.zeros((N))
for i in range(N):
    r[i] = lk[1, i] / lk[0, i]
idx = np.argsort(-r)
with open("all_word_map.txt", "r", encoding="utf-8") as f:
    content = f.read()
lines = re.split("\n", content, flags=re.S|re.M)
idx2word = {}
word2idx = {}
for l in lines:
    l = l.strip()
    if l.strip() == "":
        continue
    [word, ix] = re.split("\t", l)
    idx2word[int(ix)-1] = word
    word2idx[word] = int(ix) - 1
for i in range(10):
    ix = idx[i]
    word = idx2word[ix]
    score = r[ix]
    print("{}, {}".format(word, score))
    
# for b)
# prior:
[n_h, _] = list(ham_test.shape)
[n_s, _] = list(spam_test.shape)
lk0 = np.expand_dims(lk[0], axis=0)
lk0 = np.repeat(lk0, n_h, axis=0)
lk1 = np.expand_dims(lk[1], axis=0)
lk1 = np.repeat(lk1, n_h, axis=0)
ham_test_array = ham_test.toarray()
ph = np.sum(np.log(lk0) * ham_test_array, axis=1) + np.log(n_h / (n_h + n_s))

ps = np.sum(np.log(lk1) * ham_test_array, axis=1) + np.log(n_s / (n_h + n_s))

msk = ph > ps

tp = 0
fp = 0
tn = 0
fn = 0
tn += np.sum(msk)
fp += np.sum(~msk)
lk0 = np.expand_dims(lk[0], axis=0)
lk0 = np.repeat(lk0, n_s, axis=0)
lk1 = np.expand_dims(lk[1], axis=0)
lk1 = np.repeat(lk1, n_s, axis=0)

spam_test_array = spam_test.toarray()
ph = np.sum(np.log(lk0) * spam_test_array, axis=1) + np.log(n_h / (n_h + n_s))
ps = np.sum(np.log(lk1) * spam_test_array, axis=1) + np.log(n_s / (n_h + n_s))
msk = ph > ps
fn += np.sum(msk)
tp += np.sum(~msk)
precision = tp / (tp + fp + 1e-8)
recall = tp / (tp + fn + 1e-8)
accuracy = (tp + tn) / (tp + tn + fp + fn)
print("precision: {}".format(precision))
print("recall: {}".format(recall))
print("accuracy: {}".format(accuracy))

# end answer

nbsp, 1325.1002358991152
viagra, 1249.5763882571969
pills, 1101.9615951389017
cialis, 847.9268348888121
voip, 837.6281283921868
php, 768.9700850813518
meds, 672.8488244461829
computron, 652.2514114529324
sex, 614.4894876319731
width, 518.3682269968041
